# Project Proposal (Matt)

The ETL Project involved performing an Extract-Transform-Load (ETL) process on airline flight performance data and customer tweet data from February 2015. The members of the group are Sedra Kurdi, Myles Bridges, Natalie Myers, and Matthew Kennedy. The datasets chosen were found at https://www.kaggle.com/usdot/flight-delays and https://www.kaggle.com/crowdflower/twitter-airline-sentiment, and were in CSV and SQLite format respectively. The approach was to use the SQLAlchemy and Pandas modules in Python to extract and transform the data. SQLAlchemy was then used to load the transformed data into a PostgreSQL database.

# Project Code

## Data Extraction

### Flight Data (Myles & Sedra)

MK NOTES
1. Flight Data - Columns to Drop: 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'
2. Flight Data - Only Keep Rows with Following Airline Codes: 'UA', 'AA', 'US', 'WN', 'DL', 'VX'
3. Flight Data - Add Column for Total Delay (Add Together Departure & Arrival Delay Values)
4. Flight Data - Combine Date Value Columns into Single Date Column
5. Airline Data - Only Keep Rows with Following Airline Codes: 'UA', 'AA', 'US', 'WN', 'DL', 'VX'
6. Airport Data - Columns to Drop: 'LATITUDE', 'LONGITUDE'

In [ ]:
#ADD CODE HERE

### Tweet Data (Natalie)

MK NOTES
1. Columns to Drop: 'negativereason', 'negativereason_confidence', 'airline_sentiment_gold', 'negativereason_gold', 'tweet_coord', 'tweet_location', 'user_timezone'
2. Convert Date Column to Only Include Date & Not Time
3. Convert Airline Name to Airline Code

In [ ]:
#ADD CODE HERE

## Data Transformation

### Flight Data (Myles & Sedra)

MK NOTES
1. Airport Data - No Changes
2. Airline Data - No Changes
3. Flight Data - Use Group By & Filtering to Create Individual Data Frames for Delayed/Early/Cancelled/Diverted Flights for Each Airline (AIRLINE CODE MUST BE IN EACH DATAFRAME)

In [ ]:
#ADD CODE HERE

### Tweet Data (Natalie)

MK NOTES
1. Use Group By & Filtering to Create Data Frames of Positive/Negative/Neutral Tweets for Each Airline (AIRLINE CODE MUST BE IN EACH DATAFRAME)

In [ ]:
#ADD CODE HERE

## Data Loading (Matt)

MK Notes
1. Create ERD for Completed Database
2. Create Database in PGAdmin
3. Use SQL Alchemy to Load Dataframes into Tables & Relate by Keys

In [ ]:
#ADD CODE HERE

# Project Analysis (Matt)

The purpose of this project was to perform an Extract-Transform-Load (ETL) process on Feruary 2015 airline flight performance and customer tweets, based on datasets obtained from Kaggle. The SQLAlchemy and Pandas modules in Python were used to import, clean, and process the data, while the former was used to output the transformed data into an SQL database. Discussed below are the specific actions taken to perform each step of the ETL process.

## Data Extraction

The extraction phase of the ETL process included importing and inspecting the data, with the latter including the identification all issues and inconsistencies with the source data. To begin, there were four files that needed to be imported, which included three CSV files and one SQLite database file. A Pandas data frame was created for each of the former, which contained flight, airline, and airport information respectively. Before importing the latter, a connection was established through the SQLite engine to the database file and the table name was exported. The tweet information table was then extracted from the database directly to a Pandas data frame for inspection.

With the import process complete, the next step was to thoroughly inspect each of the data frames. Upon the completion of this process, several issues and inconsistencies were found. First, the date range of the customer tweet data was narrower than the date range of the flight performance data. Second, several columns in the flight information and tweet information data frames were missing large amounts of data. Third, the date and time values were not consistent between the flight and tweet information data frames. Fourth, the tweet information data frame contained fewer airlines than the flight information data frame. Finally, the latitude and longitude information for certain airports was incomplete in the airport information data frame.

## Data Transformation

Having established the issues with the imported data, the first step in the transformation phase was to clean each of the data frames. This was a multi-step process for each data frame, starting with the flight information data frame. TBD

## Data Loading

TBD